# Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

### List dependencies versions

In [2]:
!python --version

Python 3.6.3


In [34]:
import keras; print("keras:", keras.__version__)
import tensorflow; print("tensorflow:", tensorflow.__version__)
import numpy; print("numpy:", numpy.__version__)

keras: 2.0.9
tensorflow: 1.3.0
numpy: 1.12.1


### Import Modules

In [3]:
import collections

import helper
import numpy as np
import project_tests as tests

from typing import List, Dict, Tuple, Sequence

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15390920429200236532
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357564416
locality {
  bus_id: 1
}
incarnation: 9112511253615413743
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [5]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [8]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    # Note 1: No need to create a for loop to tokenize each sentence,
    # fit_on_texts() can handle a list of sentences.
    # Note 2: fit_on_texts() is used to build word_index by doing uique word to number mapping.
    # This mapping is later used to generate the matrix.
    lang_tokenizer = Tokenizer()
    tensor = lang_tokenizer.fit_on_texts(x)
    tensor = lang_tokenizer.texts_to_sequences(x)
    return tensor, lang_tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index, '\n')

for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21} 

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))
    print('  Output length: {}\n'.format(len(pad_sent)))

# Test that all of the sequences have the same length
def get_max_length(sequence: list) -> int:
    """Returns the length of the longest sequence"""
    return max(len(l) for l in test_pad)

assert all(len(l) == get_max_length(test_pad) for l in test_pad), \
    "All of the sequences don't have the same length"

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
  Output length: 10

Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
  Output length: 10

Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]
  Output length: 10



### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [12]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers    
    learning_rate = 1e-3

    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


tests.test_simple_model(simple_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# Convert 2d array to 3d if going from input_seq straight to GRU RNN
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1)) 


# Understand the input dimension
print("tmp_x.shape: ", tmp_x.shape)
print("tmp_x.shape[1:]: ", tmp_x.shape[1:])
print("Input(tmp_x.shape[1:]): ", Input(tmp_x.shape[1:]))
print("preproc_french_sentences.shape[-2]: ", preproc_french_sentences.shape[-2])
print("tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1)).shape): ", tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1)).shape)


# Train the neural network
print('Loading Model:')
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)


print('Model Summary:')
simple_rnn_model.summary()


print('\nTraining Model:')
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


# Print prediction(s)
print('Sample Prediction:')
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

tmp_x.shape:  (137861, 21, 1)
tmp_x.shape[1:]:  (21, 1)
Input(tmp_x.shape[1:]):  Tensor("input_2:0", shape=(?, 21, 1), dtype=float32)
preproc_french_sentences.shape[-2]:  21
tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1)).shape):  (137861, 21, 1)
Loading Model:
Model Summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_2 (GRU)                  (None, 21, 64)            12672     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 344)           22360     
_________________________________________________________________
activation_2 (Activation)    (None, 21, 344)           0         
Total params: 35,032
Trainable params: 35,032
Non-trainable params: 0
________________________________________________

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [13]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3

    input_seq = Input(input_shape[1:])
    embedding_layer = Embedding(input_dim=english_vocab_size, output_dim=128)(input_seq)
    rnn = GRU(64, return_sequences=True)(embedding_layer)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)


# TODO: Reshape the input
# Embedding() takes in a 2D tensor and returns a 3D tensor (batch_size, input_length, output_dim),
# so we don't need to reshape it.
tmp_x = pad(preproc_english_sentences, length=max_french_sequence_length)


# TODO: Train the neural network
print('Loading model:')
embedding_model = embed_model(
    input_shape=tmp_x.shape,
    output_sequence_length=max_french_sequence_length,
    english_vocab_size=english_vocab_size,
    french_vocab_size=french_vocab_size)


print('Model Summary:')
embedding_model.summary()


print('\nTraining Model:')
embedding_model.fit(x=tmp_x, y=preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


# TODO: Print prediction(s)
print('Sample Prediction:')
print(logits_to_text(embedding_model.predict(tmp_x[:1])[0], french_tokenizer))


Loading model:
Model Summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 21, 128)           25472     
_________________________________________________________________
gru_4 (GRU)                  (None, 21, 64)            37056     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 344)           22360     
_________________________________________________________________
activation_4 (Activation)    (None, 21, 344)           0         
Total params: 84,888
Trainable params: 84,888
Non-trainable params: 0
_________________________________________________________________

Training Model:
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [14]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3

    input_seq = Input(input_shape[1:])
    rnn = GRU(128, return_sequences=True)
    bd = Bidirectional(rnn)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(bd)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


tests.test_bd_model(bd_model)


# TODO: Train the neural network
print('Loading Model:')
bd_model = embed_model(
    input_shape=tmp_x.shape,
    output_sequence_length=max_french_sequence_length,
    english_vocab_size=english_vocab_size,
    french_vocab_size=french_vocab_size)


print('Model Summary:')
bd_model.summary()


print('\nTraining Model:')
bd_model.fit(x=tmp_x, y=preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


# TODO: Print prediction(s)
print('Sample Prediction:')
print(logits_to_text(bd_model.predict(tmp_x[:1])[0], french_tokenizer))


Loading Model:
Model Summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 21, 128)           25472     
_________________________________________________________________
gru_6 (GRU)                  (None, 21, 64)            37056     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 21, 344)           22360     
_________________________________________________________________
activation_6 (Activation)    (None, 21, 344)           0         
Total params: 84,888
Trainable params: 84,888
Non-trainable params: 0
_________________________________________________________________

Training Model:
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [29]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    batch = 256
    learning_rate = 1e-3

    input_seq = Input(input_shape[1:])
    rnn = LSTM(batch, return_sequences=False, dropout=0.25)(input_seq)
    rv = RepeatVector(output_sequence_length)(rnn)
    rnn = LSTM(batch, return_sequences=True, dropout=0.1)(rv)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model    


# Run tests
tests.test_encdec_model(encdec_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))


# TODO: Train the neural network
encdec_model= encdec_model(tmp_x.shape, 
                           preproc_french_sentences.shape[1], 
                           english_vocab_size, 
                           french_vocab_size)
encdec_model.summary()
encdec_model.fit(tmp_x, 
                 preproc_french_sentences, 
                 batch_size=1024, 
                 epochs=20, 
                 validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        (None, 21, 1)             0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 256)               264192    
_________________________________________________________________
repeat_vector_16 (RepeatVect (None, 21, 256)           0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 21, 256)           525312    
_________________________________________________________________
time_distributed_19 (TimeDis (None, 21, 344)           88408     
_________________________________________________________________
activation_18 (Activation)   (None, 21, 344)           0         
Total params: 877,912
Trainable params: 877,912
Non-trainable params: 0
_________________________________________________________________
Trai

In [30]:
# Printing prediction(s)
print(logits_to_text(encdec_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois agréable en l' mais il est est en juin <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [16]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 5e-3
    units = 256

    input_seq = Input(input_shape[1:])
    embedding_layer = Embedding(input_dim=english_vocab_size, output_dim=256, input_length=input_shape[1])(input_seq)
    bd1 = Bidirectional(LSTM(units))(embedding_layer)
    pad = RepeatVector(output_sequence_length)(bd1)
    rnn = LSTM(units, return_sequences=True)(pad)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


tests.test_model_final(model_final)


# Pad sequence
print("Padding input sequence")
tmp_x = pad(preproc_english_sentences)


# TODO: Train the final model
hybrid_model = model_final(
    input_shape=tmp_x.shape,
    output_sequence_length=max_french_sequence_length,
    english_vocab_size=english_vocab_size,
    french_vocab_size=french_vocab_size)
print('Loaded Model')

print('\nModel Summary:')
hybrid_model.summary()

Padding input sequence
Loaded Model

Model Summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 15)                0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 15, 256)           50944     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               1050624   
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 21, 256)           787456    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 21, 344)           88408     
_________________________________________________________________
activation_8 (Activation

## Prediction (IMPLEMENTATION)

In [17]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    
    # Pad sequence
    print("Padding input sequence")
    tmp_x = pad(x)
    print("tmp_x.shape (after padding): ", tmp_x.shape)
    print("y.shape[1]: ", y.shape[1])
    
    model = model_final(
        input_shape=tmp_x.shape,
        output_sequence_length=max_french_sequence_length,
        english_vocab_size=english_vocab_size,
        french_vocab_size=french_vocab_size)
    
    print('\nTraining Model')
    model.fit(x=tmp_x, y=y, batch_size=1024, epochs=10, validation_split=0.2)

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'

    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))


    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Padding input sequence
tmp_x.shape (after padding):  (137861, 15)
y.shape[1]:  21

Training Model
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 37s 335us/step - loss: 2.4102 - acc: 0.4886 - val_loss: nan - val_acc: 0.5820
Epoch 2/10
110288/110288 [==============================] - 36s 324us/step - loss: 1.2820 - acc: 0.6538 - val_loss: nan - val_acc: 0.7116
Epoch 3/10
110288/110288 [==============================] - 36s 324us/step - loss: 0.8985 - acc: 0.7419 - val_loss: nan - val_acc: 0.7830
Epoch 4/10
110288/110288 [==============================] - 36s 324us/step - loss: 0.6316 - acc: 0.8122 - val_loss: nan - val_acc: 0.8500
Epoch 5/10
110288/110288 [==============================] - 36s 324us/step - loss: 0.3900 - acc: 0.8877 - val_loss: nan - val_acc: 0.9177
Epoch 6/10
110288/110288 [==============================] - 36s 324us/step - loss: 0.2397 - acc: 0.9371 - val_loss: nan - val_acc: 0.9449
Epoch 7/10
110288/11028

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?

## Resources Used

Keras Docs:

- https://keras.io/api/layers/recurrent_layers/bidirectional/
- https://keras.io/examples/nlp/bidirectional_lstm_imdb/
- https://keras.io/api/layers/recurrent_layers/lstm/
- https://keras.io/api/layers/core_layers/embedding/
- https://keras.io/api/layers/core_layers/dense/

Forums that helped with troubleshooting:

- https://stackoverflow.com/questions/41602258/tokenizer-error-in-keras-pipeline
- https://thispointer.com/python-check-if-all-elements-in-a-list-are-same-or-matches-a-condition/
- https://stackoverflow.com/questions/43010339/python-tensorflow-input-to-reshape-is-a-tensor-with-92416-values-but-the-re
- https://machinelearningmastery.com/encoder-decoder-models-text-summarization-keras/
- "Understanding sequences left post-tokenization+padding versus post-preprocessing" https://knowledge.udacity.com/questions/326424
- "Where to begin with simple_model in Deciphering Code with Character-Level RNN" https://knowledge.udacity.com/questions/326428
- "Not understanding why my bidirectional model throws an AttributeError" https://knowledge.udacity.com/questions/329651
- "final predictions: wrong shape" https://knowledge.udacity.com/questions/270706